In [1]:
import torch
from modules import SConv2d, SLinear, SModel, SReLU
from models import SCrossEntropyLoss

In [30]:
fc1 = SLinear(3,3)
fc1.op.weight.data = torch.Tensor(list(range(1,10))).view(fc1.op.weight.shape) * 0.1
fc1.op.bias.data = torch.zeros_like(fc1.op.bias)
fc2 = SLinear(3,3)
fc2.op.weight.data = torch.Tensor(list(range(9,0,-1))).view(fc2.op.weight.shape) * 0.1
fc2.op.weight.data[0,0] = 2
fc2.op.bias.data = torch.zeros_like(fc2.op.bias)
relu = SReLU()
criteria = SCrossEntropyLoss()
optimizer = torch.optim.SGD(list(fc1.parameters()) + list(fc2.parameters()), 0)

optimizer.zero_grad()
x = torch.Tensor([[1,2,3]])
xS = torch.zeros_like(x)
labels = torch.LongTensor([0])

x1, xS1 = fc1(x, xS)
x1.retain_grad()
xS1.retain_grad()
x2, xS2 = relu(x1, xS1)
x2.retain_grad()
xS2.retain_grad()
x3, xS3 = fc2(x2, xS2)
x3.retain_grad()
xS3.retain_grad()

loss = criteria(x3, xS3, labels)
loss.backward()

In [47]:
x3SG = torch.Tensor([0.0123928,-0.0117341,-0.000658689,-0.0117341,0.011742,-7.92681E-6,-0.000658689,-7.92681E-6,0.000666616] ).view(3,3)
x1SG = torch.Tensor([0.0249031,0.00560088,0.00560088,0.00560088,0.00129391,0.00129391,0.00560088,0.00129391,0.00129391]).view(3,3)

In [55]:
fc1WSG = torch.zeros(9,9)
IN = x.view(-1)
for i in range(3):
    for j in range(3):
        for k in range(3):
            for m in range(3):
                # wij * wkm
                fc1WSG[i*3+j, k*3+m] = x1SG[j,m] * IN[i] * IN[k] 

In [57]:
l = []
for i in range(9):
    l.append(fc1WSG[i,i].item())
torch.Tensor(l).view(3,3)

tensor([[0.0249, 0.0013, 0.0013],
        [0.0996, 0.0052, 0.0052],
        [0.2241, 0.0116, 0.0116]])

In [58]:
fc1WSG

tensor([[0.0249, 0.0056, 0.0056, 0.0498, 0.0112, 0.0112, 0.0747, 0.0168, 0.0168],
        [0.0056, 0.0013, 0.0013, 0.0112, 0.0026, 0.0026, 0.0168, 0.0039, 0.0039],
        [0.0056, 0.0013, 0.0013, 0.0112, 0.0026, 0.0026, 0.0168, 0.0039, 0.0039],
        [0.0498, 0.0112, 0.0112, 0.0996, 0.0224, 0.0224, 0.1494, 0.0336, 0.0336],
        [0.0112, 0.0026, 0.0026, 0.0224, 0.0052, 0.0052, 0.0336, 0.0078, 0.0078],
        [0.0112, 0.0026, 0.0026, 0.0224, 0.0052, 0.0052, 0.0336, 0.0078, 0.0078],
        [0.0747, 0.0168, 0.0168, 0.1494, 0.0336, 0.0336, 0.2241, 0.0504, 0.0504],
        [0.0168, 0.0039, 0.0039, 0.0336, 0.0078, 0.0078, 0.0504, 0.0116, 0.0116],
        [0.0168, 0.0039, 0.0039, 0.0336, 0.0078, 0.0078, 0.0504, 0.0116, 0.0116]])